In [12]:
import numpy as np
import plotly.graph_objects as go
from sklearn.decomposition import PCA
from itertools import combinations

# ============================================================
# 1. Generate 5D synthetic dataset
# ============================================================

# Set reproducible random seed
np.random.seed(42)

# Mean of the 5D Gaussian distribution
mean = np.zeros(5)

# Covariance matrix (defines correlation structure)
cov = np.array([
    [1.0, 0.8, 0.3, 0.0, 0.2],
    [0.8, 1.5, 0.4, 0.1, 0.3],
    [0.3, 0.4, 1.2, 0.3, 0.5],
    [0.0, 0.1, 0.3, 0.8, 0.2],
    [0.2, 0.3, 0.5, 0.2, 1.0]
])

# Generate 600 random points in 5D from the distribution
X = np.random.multivariate_normal(mean, cov, size=600)


# ============================================================
# 2. Perform PCA on the 5D dataset
# ============================================================

# Initialize PCA to extract all 5 components
pca = PCA(n_components=5)

# Transform the data into PCA space
# X_pca[i] = coordinates of point i in the new principal-component basis
X_pca = pca.fit_transform(X)

# PCA components (eigenvectors of covariance matrix)
# shape = (5 principal components) x (5 dimensions)
components = pca.components_

# Scaling factor to visually enlarge principal axes in plots
axis_scale = 5


# ============================================================
# 3. FUNCTION: Plot a 3D scatter for a given triplet of PCs
#    Also draw principal axes projected into this 3D space
# ============================================================

def plot_3d_triplet(indices):
    """
    Plot a 3D scatter using three selected principal components.
    Also draw all 5 PCA axes projected into this 3D coordinate system.
    
    Parameters:
        indices: tuple of 3 integers → which PCs to plot, e.g. (0,1,2)
    """
    
    # Unpack the PC indices
    a, b, c = indices

    # Create a new 3D Plotly figure
    fig = go.Figure()

    # ------------------------------------------------------------
    # 3D scatter of the projected dataset
    # ------------------------------------------------------------
    fig.add_trace(go.Scatter3d(
        x=X_pca[:, a],       # x-axis = PC_a values
        y=X_pca[:, b],       # y-axis = PC_b values
        z=X_pca[:, c],       # z-axis = PC_c values
        mode="markers",      # plotted as points
        marker=dict(
            size=3,
            color=X_pca[:, a],  # color encoded by PC_a
            colorscale="Viridis"
        ),
        name="Data"
    ))

    # ------------------------------------------------------------
    # Draw the principal axes in this 3D plot
    # Each PCA axis is an eigenvector in 5D.
    # We project it onto the selected 3 PCs.
    # ------------------------------------------------------------
    for i, comp in enumerate(components):

        # Project eigenvector "comp" into the selected PCA dimensions
        vec = comp[[a, b, c]] * axis_scale

        # Draw arrow from origin (0,0,0) to the vector
        fig.add_trace(go.Scatter3d(
            x=[0, vec[0]],
            y=[0, vec[1]],
            z=[0, vec[2]],
            mode="lines+text",
            line=dict(width=8),
            text=[None, f"PC{i+1}"],      # label the arrow tip
            textposition="top center",
            name=f"PC{i+1}"
        ))

    # ------------------------------------------------------------
    # Configure layout, titles, axes labels
    # ------------------------------------------------------------
    fig.update_layout(
        title=f"3D PCA Projection (PC{a+1}, PC{b+1}, PC{c+1}) with Principal Axes",
        scene=dict(
            xaxis_title=f"PC{a+1}",
            yaxis_title=f"PC{b+1}",
            zaxis_title=f"PC{c+1}",
        ),
        width=900,
        height=700
    )

    # Display the plot
    fig.show()



# ============================================================
# 4. FUNCTION: Plot 2D projections for a triplet of PCs
#    Creates (PCa vs PCb), (PCa vs PCc), (PCb vs PCc)
# ============================================================

def plot_2d_pairs(indices):
    """
    Given 3 PCA indices, create the 3 possible 2D projection plots.
    
    Parameters:
        indices: tuple of 3 integers
    """

    # Unpack the PCA indices
    a, b, c = indices

    # All 2D combinations from the triplet
    pairs = [(a, b), (a, c), (b, c)]

    # ------------------------------------------------------------
    # Loop over each 2D pair and create a scatter plot
    # ------------------------------------------------------------
    for (x, y) in pairs:

        # Create a new 2D figure
        fig = go.Figure()

        # ----------------------
        # Scatter points
        # ----------------------
        fig.add_trace(go.Scatter(
            x=X_pca[:, x],
            y=X_pca[:, y],
            mode="markers",
            marker=dict(
                size=5,
                color=X_pca[:, x],
                colorscale="Plasma"
            ),
            name="Data"
        ))

        # ----------------------
        # Add PCA axes projected into this 2D plane
        # ----------------------
        for i, comp in enumerate(components):

            # Select only the dimensions corresponding to this pair
            vec = comp[[x, y]] * axis_scale

            fig.add_trace(go.Scatter(
                x=[0, vec[0]],
                y=[0, vec[1]],
                mode="lines+text",
                text=[None, f"PC{i+1}"],
                textposition="top center",
                line=dict(width=4),
                name=f"PC{i+1}"
            ))

        # ----------------------
        # Layout and titles
        # ----------------------
        fig.update_layout(
            title=f"2D PCA Projection: PC{x+1} vs PC{y+1}",
            xaxis_title=f"PC{x+1}",
            yaxis_title=f"PC{y+1}",
            width=600,
            height=500
        )

        # Display the figure
        fig.show()



# ============================================================
# 5. Generate all combinations of 3 PCs out of 5
# ============================================================

# List of all 10 combinations of (PCi, PCj, PCk)
triplets = list(combinations(range(5), 3))


# ============================================================
# 6. Loop over each triplet:
#       → Show 3D plot with axes
#       → Show 2D projections
# ============================================================

for triple in triplets:
    print(f"\n=== Showing PCA components: PC{triple[0]+1}, PC{triple[1]+1}, PC{triple[2]+1} ===\n")

    # 3D PCA space + axes
    plot_3d_triplet(triple)

    # The 3 pairwise 2D projections
    plot_2d_pairs(triple)



=== Showing PCA components: PC1, PC2, PC3 ===




=== Showing PCA components: PC1, PC2, PC4 ===




=== Showing PCA components: PC1, PC2, PC5 ===




=== Showing PCA components: PC1, PC3, PC4 ===




=== Showing PCA components: PC1, PC3, PC5 ===




=== Showing PCA components: PC1, PC4, PC5 ===




=== Showing PCA components: PC2, PC3, PC4 ===




=== Showing PCA components: PC2, PC3, PC5 ===




=== Showing PCA components: PC2, PC4, PC5 ===




=== Showing PCA components: PC3, PC4, PC5 ===

